# Test File for Testing DFT and IDFT Algorithms

In [1]:
%reset -f
import gc
gc.collect()

0

In [2]:
import numpy as np
from scipy.fft import fft, ifft
import tensorflow as tf

np.random.seed(42)

n = 16
q = 5
num_samples = 3

dataset = np.random.randint(0, q, size=(num_samples, n))
print(dataset)

[[3 4 2 4 4 1 2 2 2 4 3 2 4 1 3 1]
 [3 4 0 3 1 4 3 0 0 2 2 1 3 3 2 3]
 [3 0 2 4 2 4 0 1 3 0 3 1 1 0 1 4]]


In [3]:
n1=n//2

zeta = np.exp(-2j * np.pi / n)
Dn = np.array([zeta**k for k in range(n1)])

print(Dn)
print(np.conj(Dn))

[ 1.00000000e+00+0.j          9.23879533e-01-0.38268343j
  7.07106781e-01-0.70710678j  3.82683432e-01-0.92387953j
 -2.22044605e-16-1.j         -3.82683432e-01-0.92387953j
 -7.07106781e-01-0.70710678j -9.23879533e-01-0.38268343j]
[ 1.00000000e+00-0.j          9.23879533e-01+0.38268343j
  7.07106781e-01+0.70710678j  3.82683432e-01+0.92387953j
 -2.22044605e-16+1.j         -3.82683432e-01+0.92387953j
 -7.07106781e-01+0.70710678j -9.23879533e-01+0.38268343j]


# **DFT**

### Brute Force DFT

$$
    X[k] = \sum_{n=0}^{N-1} x[n] e^{-j \frac{2\pi}{N} kn}, \quad k = 0,1,\dots, N-1
$$

In [4]:
def dft_brute_force(x, N):
    X = np.zeros(N, dtype=complex)

    for k in range(N):
        for n in range(N):
            X[k] += x[n] * np.exp(-2j * np.pi * k * n / N)

    return X

### Recursive Radix-2 DFT

$$
F_n = P_n
\begin{bmatrix}
F_{n/2} & 0 \\
0 & F_{n/2}
\end{bmatrix}
H_n
$$

$$
H_n =
\begin{bmatrix}
I_{\frac{n}{2}} & I_{\frac{n}{2}} \\
D_n & -D_n
\end{bmatrix}
\quad \text{ where } \quad
D_n = \operatorname{diag}(1, \zeta, \zeta^2, \dots, \zeta^{\frac{n}{2} - 1}), \quad \text{where } \zeta = e^{-2\pi i/n}
$$

In [5]:
# using matmul

def dft_matmul(u, n):
    n1 = n // 2
    if n == 2:
        return np.array([u[0] + u[1], u[0] - u[1]])

    zeta = np.exp(-2j * np.pi / n)
    Dn = np.diag([zeta**k for k in range(n1)])
    Hn = np.block([[np.eye(n1), np.eye(n1)],
                   [Dn, -Dn]])

    p = np.dot(Hn, u)
    # p = Hn @ u

    s1 = dft(p[:n1], n1)
    s2 = dft(p[n1:], n1)

    interleaved = np.empty((len(s1) + len(s2)), dtype=complex)  # data type - complex
    interleaved[0::2] = s1  # even indices with s1
    interleaved[1::2] = s2  # odd indices with s2

    y = interleaved

    return y

In [6]:
def dft(u, n):
    n1 = n // 2
    if n == 2:
        return np.array([u[0] + u[1], u[0] - u[1]])
    else:
        zeta = np.exp(-2j * np.pi / n)
        Dn = np.array([zeta**k for k in range(n1)])
        p_first = u[:n1] + u[n1:]
        p_second = Dn * (u[:n1] - u[n1:])
        s1 = dft(p_first, n1)
        s2 = dft(p_second, n1)
        y = np.empty(n, dtype=complex)
        y[0::2] = s1
        y[1::2] = s2
        return y

# **IDFT**

### Brute Force IDFT

In [7]:
def idft_brute_force(X, N):
    x = np.zeros(N, dtype=complex)

    for n in range(N):
        for k in range(N):
            x[n] += X[k] * np.exp(2j * np.pi * k * n / N)

    return x / N

### Recursive Radix-2 IDFT

$$
F_n^{*} = \frac{1}{2} H_n^*
\begin{bmatrix}
F_{n/2}^{*} & 0 \\
0 & F_{n/2}^{*}
\end{bmatrix}
P_n^T
$$

$$
H_n =
\begin{bmatrix}
I_{\frac{n}{2}} & I_{\frac{n}{2}} \\
D_n & -D_n
\end{bmatrix}
\quad \text{ where } \quad
D_n = \operatorname{diag}(1, \zeta, \zeta^2, \dots, \zeta^{\frac{n}{2} - 1}), \quad \text{where } \zeta = e^{-2\pi i/n}
$$

In [8]:
# using matmul

def idft_matmul(y, n):
    n1 = n// 2

    if n == 2:
        return np.array([y[0] + y[1], y[0] - y[1]]) / 2

    elif n >= 4:
        q = np.concatenate((y[0:n:2], y[1:n:2]))

        b1 = idft(q[:n1], n1)
        b2 = idft(q[n1:], n1)

        b_out = np.concatenate((b1, b2))

        zeta = np.exp(-2j * np.pi / n)
        Dn = np.diag([zeta**k for k in range(n1)])
        Hn = np.block([[np.eye(n1), np.eye(n1)],
                       [Dn, -Dn]])

        Hn_conj = np.conjugate(Hn).T

        z1 = np.dot(Hn_conj, b_out)

        out = z1 / 2

        # real = np.real(out)
        # img = np.imag(out)

        return out # Scale down since IDFT requires normalization

    # note: i tried to uncomment np.real(out) and return only the real value. it doesn't work.
    # rather run the algorithm as it is and then take the np.real() of the final result
    # this might be because this is a recursive function and the function implicitly passes the value as a complex number in the next call.

In [9]:
def idft(y, n):
    n1 = n// 2

    if n == 2:
        return np.array([y[0] + y[1], y[0] - y[1]]) / 2

    elif n >= 4:
        q = np.concatenate((y[0:n:2], y[1:n:2]))

        B1 = idft(q[:n1], n1)
        B2 = idft(q[n1:], n1)

        b_out = np.concatenate((B1, B2))

        zeta = np.exp(-2j * np.pi / n)
        Dn = np.array([zeta**k for k in range(n1)])

        # z1 = np.concatenate([B1 + Dn * B2, B1 - Dn * B2], axis=0)
        z1 = np.concatenate([B1 + np.conj(Dn) * B2, B1 - np.conj(Dn) * B2], axis=0)

        out = z1 / 2

        return out

### Test Cases

Encoding (DFT)

In [10]:
for i, message in enumerate(dataset):
    print(f"\nOriginal Message {i+1}: {message}")

    brute = dft_brute_force(message, n)
    print("DFT (brute):     ", np.round(brute, 4))

    scipy_fft = fft(message)
    print("DFT (scipy fft): ", np.round(scipy_fft, 4))

    matmul = dft_matmul(message, n)
    print("DFT (matmul):    ", np.round(matmul, 4))

    rec = dft(message, n)
    print("DFT (optimized): ", np.round(rec, 4))

    print("Match brute & scipy:", np.allclose(brute, scipy_fft))
    print("Match optimized DFT & scipy:", np.allclose(rec, scipy_fft))
    print("Match matmul & scipy:", np.allclose(matmul, scipy_fft))
    print("Match optimized DFT & matmul:", np.allclose(matmul, rec))


Original Message 1: [3 4 2 4 4 1 2 2 2 4 3 2 4 1 3 1]
DFT (brute):      [42.    +0.j      0.8415-0.8162j -0.8787-6.364j  -1.2304+1.2557j
  3.    -1.j      3.2304-1.5727j -5.1213-6.364j   1.1585-3.6447j
  4.    -0.j      1.1585+3.6447j -5.1213+6.364j   3.2304+1.5727j
  3.    +1.j     -1.2304-1.2557j -0.8787+6.364j   0.8415+0.8162j]
DFT (scipy fft):  [42.    -0.j      0.8415-0.8162j -0.8787-6.364j  -1.2304+1.2557j
  3.    -1.j      3.2304-1.5727j -5.1213-6.364j   1.1585-3.6447j
  4.    -0.j      1.1585+3.6447j -5.1213+6.364j   3.2304+1.5727j
  3.    +1.j     -1.2304-1.2557j -0.8787+6.364j   0.8415+0.8162j]
DFT (matmul):     [42.    +0.j      0.8415-0.8162j -0.8787-6.364j  -1.2304+1.2557j
  3.    -1.j      3.2304-1.5727j -5.1213-6.364j   1.1585-3.6447j
  4.    +0.j      1.1585+3.6447j -5.1213+6.364j   3.2304+1.5727j
  3.    +1.j     -1.2304-1.2557j -0.8787+6.364j   0.8415+0.8162j]
DFT (optimized):  [42.    +0.j      0.8415-0.8162j -0.8787-6.364j  -1.2304+1.2557j
  3.    -1.j      3.2304-

Decoding (IDFT)

In [11]:
for i, message in enumerate(dataset):
    print(f"\nOriginal Message {i+1}: {message}")

    encoded = fft(message)
    # print("Encoded (scipy FFT):", np.round(encoded, 4))

    brute_decoded = idft_brute_force(encoded, n)
    print("iDFT (brute):       ", np.round(brute_decoded, 4))

    scipy_ifft = ifft(encoded)
    print("iDFT (scipy ifft):  ", np.round(scipy_ifft, 4))

    matmul_decoded = idft_matmul(encoded, n)
    print("iDFT (matmul):      ", np.round(matmul_decoded, 4))

    rec_decoded = idft(encoded, n)
    print("iDFT (optimized):   ", np.round(rec_decoded, 4))

    # compare all decoders to original message
    print("Brute ≈ original:", np.allclose(brute_decoded, message))
    print("Optimized ≈ original:", np.allclose(rec_decoded, message))
    print("Matmul ≈ original:", np.allclose(matmul_decoded, message))
    print("Scipy ≈ original:", np.allclose(scipy_ifft, message))

    # compare among methods too
    print("Optimized ≈ Matmul:", np.allclose(matmul_decoded, rec_decoded))
    print("Brute ≈ Scipy:", np.allclose(brute_decoded, scipy_ifft))


Original Message 1: [3 4 2 4 4 1 2 2 2 4 3 2 4 1 3 1]
iDFT (brute):        [3.-0.j 4.-0.j 2.+0.j 4.-0.j 4.-0.j 1.+0.j 2.-0.j 2.+0.j 2.+0.j 4.-0.j
 3.-0.j 2.+0.j 4.-0.j 1.+0.j 3.+0.j 1.+0.j]
iDFT (scipy ifft):   [3.+0.j 4.+0.j 2.+0.j 4.+0.j 4.+0.j 1.-0.j 2.+0.j 2.-0.j 2.+0.j 4.+0.j
 3.+0.j 2.+0.j 4.+0.j 1.-0.j 3.-0.j 1.-0.j]
iDFT (matmul):       [3.+0.j 4.-0.j 2.+0.j 4.-0.j 4.+0.j 1.+0.j 2.-0.j 2.+0.j 2.+0.j 4.-0.j
 3.+0.j 2.-0.j 4.+0.j 1.+0.j 3.+0.j 1.+0.j]
iDFT (optimized):    [3.+0.j 4.-0.j 2.-0.j 4.-0.j 4.+0.j 1.+0.j 2.-0.j 2.+0.j 2.+0.j 4.-0.j
 3.+0.j 2.-0.j 4.+0.j 1.+0.j 3.+0.j 1.+0.j]
Brute ≈ original: True
Optimized ≈ original: True
Matmul ≈ original: True
Scipy ≈ original: True
Optimized ≈ Matmul: True
Brute ≈ Scipy: True

Original Message 2: [3 4 0 3 1 4 3 0 0 2 2 1 3 3 2 3]
iDFT (brute):        [ 3.+0.j  4.-0.j  0.-0.j  3.-0.j  1.+0.j  4.-0.j  3.+0.j -0.+0.j  0.+0.j
  2.-0.j  2.+0.j  1.+0.j  3.-0.j  3.-0.j  2.-0.j  3.-0.j]
iDFT (scipy ifft):   [ 3.+0.j  4.-0.j  0.+0.j  3.+0.

Notes: take the real part of the output